In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path

In [ ]:
rna_count_folder = '../Data/RNAscoring_dev_data/RNA_counts/' #path to RNA counts folder
dna_count_folder = '../Data/RNAscoring_dev_data/DNA_counts/' #paths to DNA counts folder (currently the same, but will code as if they are not)

In [ ]:
def read_counts(rna_counts, dna_counts):
    rna_path = Path(rna_counts) #Creates path object to RNA counts
    dna_path = Path(dna_counts) #Creates path object to DNA counts
    
    rna_outputs = list(rna_path.glob('*.tsv')) #Gets list of RNA count files
    dna_outputs = list(dna_path.glob('*.tsv')) #Gets list of DNA count files
    
    print(rna_outputs)

In [ ]:
def main():
    read_counts(rna_count_folder, dna_count_folder)

In [ ]:
main()